<a href="https://colab.research.google.com/github/micheleguidaa/robust-organ-scheduler/blob/main/robust_organ_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pianificazione Robusta dell'Orario di Prelievo in Interventi di Trapianto

**Studente:** Michele Guida  
**Corso:** Decision Support Systems  
**Repository:** [github.com/micheleguidaa/robust-organ-scheduler](https://github.com/micheleguidaa/robust-organ-scheduler)

## 1. Definizione del Problema

Il progetto affronta la scelta dell'orario di inizio ($t_{start}$) di un intervento di prelievo multi-organo da un singolo donatore.
L'obiettivo è minimizzare i ritardi di partenza dei mezzi di trasporto assegnati ai diversi organi, tenendo conto
della diversa importanza clinica e dell'incertezza sulla durata dell'intervento.

### Modello Matematico

Sia $\mathbf{T}$ l'insieme discreto degli orari di inizio possibili e $\mathbf{D} = \{d_{min}, d_{med}, d_{max}\}$ l'insieme delle possibili durate dell'intervento.

Definiamo il **tempo di attesa** $\delta_i$ per l'organo $i$ come il tempo che intercorre tra la disponibilità dell'organo (fine intervento) e la prima partenza utile del mezzo di trasporto $S_i^j$:

$$
\delta_{i}(t_{start},d) = \min_{j: S_{i}^{j} \ge t_{start}+d} (S_{i}^{j} - (t_{start}+d))
$$

La **funzione di costo** (ritardo totale pesato) per uno scenario è data da:

$$
F(t_{start},d) = \sum_{i \in \mathbf{I}} w_{i} \cdot \delta_{i}(t_{start},d)
$$

### Obiettivo: Ottimizzazione Robusta (Minimax)
Poiché la durata $d$ è incerta, adottiamo un approccio robusto che minimizza il costo nel *caso peggiore* (worst-case scenario):

$$
t_{start}^* = \arg \min_{t_{start} \in \mathbf{T}} \left( \max_{d \in \mathbf{D}} F(t_{start}, d) \right)
$$

## 2. Soluzione e Implementazione dell'Algoritmo

Proponiamo due varianti dell'algoritmo in base alle assunzioni sulla struttura dei dati di input (orari di partenza degli organi).


### 2.a Soluzione Generale (Partenze NON Ordinate)
Questa versione (`robust_organ_scheduler_unsorted`) non fa alcuna assunzione sull'ordinamento degli orari di partenza.
Per trovare la prima partenza utile, deve scansionare l'intera lista delle partenze disponibili per ogni organo.

*   **Approccio:** Scansione Lineare (Linear Scan).
*   **Complessità:** $O\left( |\mathbf{T}| \cdot |\mathbf{D}| \cdot \sum_{i=1}^{N} |\mathbf{S}_i| \right)$.
*   **Utilizzo:** Adatta quando i dati provengono da fonti non strutturate o non ordinate.


In [1]:
import numpy as np
from datetime import datetime, timedelta
from typing import Sequence, Mapping, Tuple, Optional

# ---------------------------------------------------------
# 2.a SOLUZIONE PER PARTENZE NON ORDINATE (Linear Scan)
# ---------------------------------------------------------
def robust_organ_scheduler_unsorted(
    orari_possibili_inizio_intervento: Sequence[datetime],
    durate_possibili_intervento: Sequence[timedelta],
    organi: Sequence[str],
    orari_partenza: Mapping[str, Sequence[datetime]],
    pesi: Mapping[str, float],
) -> Tuple[float, Optional[datetime], timedelta]:
    """
    Risolve il problema di scheduling robusto (minimax) restituendo:
    - valore della funzione obiettivo (float)
    - orario di inizio intervento ottimo (datetime) oppure None se non esiste soluzione
    - tempo di esecuzione dell'algoritmo (timedelta)
    """
    # Avvio del timer per misurare il tempo di esecuzione dell’algoritmo.
    inizio_esecuzione_algoritmo = datetime.now()

    # Inizializzazione del problema minimax: vogliamo minimizzare il massimo delle attese pesate
    funzione_obiettivo = np.inf
    migliore_orario_inizio_intervento = None

    # Valutazione di ogni possibile orario di inizio intervento
    for orario_inizio_intervento in orari_possibili_inizio_intervento:

        # Per questo orario consideriamo il peggior scenario tra tutte le durate
        somma_pesata_attese_max = -np.inf

        for durata_intervento in durate_possibili_intervento:
            # Calcolo della somma pesata delle attese per questa durata
            somma_pesata_attese = 0.0

            for organo in organi:
                # SCANSIONE LINEARE: Filtra tutte le partenze valide
                # Orario di fine intervento per questa durata
                orario_fine_intervento = orario_inizio_intervento + durata_intervento

                # Selezioniamo solo le partenze non antecedenti alla fine dell'intervento
                partenze_valide = [
                    orario_partenza for orario_partenza in orari_partenza[organo]
                    if orario_partenza >= orario_fine_intervento
                ]

                # Se non ci sono partenze valide l’intervento diventa impossibile → costo infinito
                if not partenze_valide:
                    somma_pesata_attese = np.inf
                    break

                # Prima partenza disponibile
                prima_partenza_utile = min(partenze_valide)

                # Tempo di attesa rispetto alla fine dell’intervento
                attesa = prima_partenza_utile - orario_fine_intervento
                attesa_minuti = attesa.total_seconds() / 60

                # Aggiunta dell’attesa pesata secondo l’importanza dell’organo
                somma_pesata_attese += pesi[organo] * attesa_minuti

            # Aggiornamento dello scenario peggiore (max) per questo t_start
            if somma_pesata_attese > somma_pesata_attese_max:
                somma_pesata_attese_max = somma_pesata_attese

        # Aggiorniamo il best t_start globale confrontando i valori worst-case ottenuti
        if somma_pesata_attese_max < funzione_obiettivo:
            funzione_obiettivo = somma_pesata_attese_max
            migliore_orario_inizio_intervento = orario_inizio_intervento

    # Fine della misurazione del tempo totale di esecuzione.
    fine_esecuzione_algoritmo = datetime.now()

    # tempo_di_esecuzione quantifica il costo computazionale dell’enumerazione.
    tempo_di_esecuzione = fine_esecuzione_algoritmo - inizio_esecuzione_algoritmo

    return funzione_obiettivo, migliore_orario_inizio_intervento, tempo_di_esecuzione



### 2.b Soluzione Ottimizzata (Partenze ORDINATE)
Questa versione (`robust_organ_scheduler_sorted`) assume che le liste degli orari di partenza $\mathbf{S}_i$ siano **già ordinate** in ordine cronologico crescente.
Sfruttando questa proprietà, possiamo utilizzare la **ricerca binaria** (Binary Search) per trovare efficientemente la prima partenza successiva alla fine dell'intervento, senza dover scansionare l'intera lista.

*   **Approccio:** Ricerca Binaria (Binary Search).
*   **Complessità:** $O\left( |\mathbf{T}| \cdot |\mathbf{D}| \cdot \sum_{i=1}^{N} \log |\mathbf{S}_i| \right)$.
*   **Vantaggio:** Riduce drasticamente il tempo di calcolo quando il numero di partenze $|\mathbf{S}_i|$ è elevato.

In [2]:
import numpy as np
import bisect
from datetime import datetime, date, timedelta
from typing import Sequence, Mapping, Tuple, Optional

# ---------------------------------------------------------
# 2.b SOLUZIONE PER PARTENZE ORDINATE (Binary Search)
# ---------------------------------------------------------
def robust_organ_scheduler_sorted(
    orari_possibili_inizio_intervento: Sequence[datetime],
    durate_possibili_intervento: Sequence[timedelta],
    organi: Sequence[str],
    orari_partenza: Mapping[str, Sequence[datetime]],
    pesi: Mapping[str, float],
) -> Tuple[float, Optional[datetime], timedelta]:
    """
    Versione che ASSUME partenze ordinate.
    Usa Binary Search (bisect) O(log N).
    """
    inizio_esecuzione = datetime.now()
    funzione_obiettivo = np.inf
    migliore_orario = None

    for orario_inizio in orari_possibili_inizio_intervento:
        somma_pesata_attese_max = -np.inf

        for durata in durate_possibili_intervento:
            somma_pesata_attese = 0.0
            orario_fine = orario_inizio + durata

            for organo in organi:
                lista_partenze = orari_partenza[organo]
                
                # RICERCA BINARIA: Trova il punto di inserimento
                # bisect_left restituisce l'indice del primo elemento >= orario_fine
                idx = bisect.bisect_left(lista_partenze, orario_fine)

                if idx < len(lista_partenze):
                    prima_partenza = lista_partenze[idx]
                    attesa_minuti = (prima_partenza - orario_fine).total_seconds() / 60
                    somma_pesata_attese += pesi[organo] * attesa_minuti
                else:
                    # Nessuna partenza valida trovata dopo la fine intervento
                    somma_pesata_attese = np.inf
                    break

            if somma_pesata_attese > somma_pesata_attese_max:
                somma_pesata_attese_max = somma_pesata_attese

        if somma_pesata_attese_max < funzione_obiettivo:
            funzione_obiettivo = somma_pesata_attese_max
            migliore_orario = orario_inizio

    tempo_esecuzione = datetime.now() - inizio_esecuzione
    return funzione_obiettivo, migliore_orario, tempo_esecuzione

# Test e Valutazione delle Prestazioni

## Obiettivo
La fase di test ha lo scopo di valutare le prestazioni delle due varianti dell’algoritmo utilizzando dataset sintetici con caratteristiche controllate.  

---

## Pesi degli Organi

Nell’esperimento sono stati assegnati pesi differenziati agli organi per riflettere la loro diversa priorità clinica.

$$
w_{\text{Cuore}} = 100,\quad
w_{\text{Polmoni}} = 90,\quad
w_{\text{Rene\,Sx}} = 20,\quad
w_{\text{Rene\,Dx}} = 20,\quad
w_{\text{Fegato}} = 20.
$$

Questi pesi vengono utilizzati nella funzione obiettivo per modulare il contributo del tempo di attesa dei singoli organi.

---

## Generazione dei Dati Sintetici

### 1. Orari di inizio intervento

Gli orari di inizio vengono generati come una sequenza crescente avente queste regole:

- `t_0 = 0`
- `t_1 ∈ [0.5 h ,1 h, 1.5 h, 2 h]`
- `t_2 ∈ (t_1 , t_1 + 0.5 h, ..., 3 h]`
- `t_3 ∈ (t_2, t_2 + 0.5 h, ..., 5 h]`

Da notare come gli orari vengano generati con passi di 30 minuti (0.5 h).

Tale sequenza soddisfa:

$$
0 = t_0 < t_1 < t_2 < t_3 \le 5\text{ h}.
$$

L’insieme dei tempi ammissibili è:

$$
\mathbf{T} = \{t_0,\; t_1,\; t_2,\; t_3\}.
$$

---

### 2. Orari di partenza degli organi

Per ogni organo $ i $, gli orari di partenza sono generati come valori uniformi nell’intervallo discreto:

$$
S_i^j \in [0,\; 0.5,\; 1,\; \ldots,\; 23.5,\; 24\text{ h}]
$$

Anche in questo caso, gli orari sono generati con passi di 30 minuti (0.5 h).

Ciascun organo randomicamente riceve un numero di orari di partenza compreso tra 10 e 35.

---

### 3. Durata dell’intervento

La durata dell’intervento è considerata incerta e può assumere uno dei seguenti valori:

$$
d \in \{2,\; 4,\; 6\}\text{ h}.
$$

---

## Protocollo di Test

Per ogni configurazione sperimentale:

- l’esperimento viene ripetuto **50 volte** con diversi dataset generati casualmente;
- per ciascun run vengono registrati e salvati:
  - i parametri di input generati,
  - il tempo di esecuzione,
  - la soluzione ottimale,
  - il tempo di attesa di ciascun organo.

Dai risultati si ricavano:

- il **tempo medio e deviazione standard di esecuzione**,  
- il **valore medio e deviazione standard de lla soluzione pesata**,  
- il **tempo medio di attesa per organo**:
- il **tempo massimo di attesa per organo**:

---

## Analisi di Sensitività sulla Densità degli Orari di Partenza

Per valutare l’effetto della riduzione della densità degli orari di partenza:

1. si ordinano i valori $ S_i^j $ per ogni organo;
2. si rimuove un orario ogni due (pattern alternato);
3. si ripete l’intero processo di test sulla nuova configurazione.

L’analisi consente di esaminare l’impatto della variazione della granularità dei dati su:

- tempo di esecuzione,
- qualità della soluzione,
- robustezza dell’algoritmo (in presenza dei pesi $ w_i $).

---


In [41]:
import random
import datetime
from typing import Sequence, List, Dict

# ---------------------------------------------------------
# CONFIGURAZIONE SPERIMENTALE
# ---------------------------------------------------------
ORGANI = ['Cuore', 'Polmoni', 'Rene_sx', 'Rene_dx', 'Fegato']

PESO_ORGANI = {
    'Cuore': 10.0,
    'Polmoni': 9.0,
    'Rene_sx': 2.0,     
    'Rene_dx': 2.0,
    'Fegato': 2.0
}

# ---------------------------------------------------------
# 1. GENERAZIONE ORARI POSSIBILI DI INIZIO INTERVENTO
# ---------------------------------------------------------
def genera_orari_possibili_inizio_intervento(
    inizio_giorno: datetime.datetime = datetime.datetime.combine(datetime.date.today(), datetime.time(0, 0))
) -> Sequence[datetime.datetime]:
    """
    Genera 4 istanti temporali t0, t1, t2, t3 per l'inizio dell'intervento.
    t0 = 0
    t1 in [0.5, 1, 1.5, 2] ore
    t2 in (t1, 3] ore
    t3 in (t2, 5] ore
    """
    
    passo = 30 # minuti
    
    # t_0 = 0
    t0_min = 0

    # t_1: scelta tra 30, 60, 90, 120 minuti
    possibili_t1 = [30, 60, 90, 120]
    t1_min = random.choice(possibili_t1)

    # t_2: strettamente maggiore di t1, fino a 180 min (3h)
    # range(start, stop, step) -> start = t1 + 30
    possibili_t2 = list(range(t1_min + passo, 180 + 1, passo))
    t2_min = random.choice(possibili_t2)

    # t_3: strettamente maggiore di t2, fino a 300 min (5h)
    possibili_t3 = list(range(t2_min + passo, 300 + 1, passo))
    t3_min = random.choice(possibili_t3)

    # Creazione lista datetime
    orari = [
        inizio_giorno + datetime.timedelta(minutes=t0_min),
        inizio_giorno + datetime.timedelta(minutes=t1_min),
        inizio_giorno + datetime.timedelta(minutes=t2_min),
        inizio_giorno + datetime.timedelta(minutes=t3_min)
    ]
    
    return orari

# ---------------------------------------------------------
# 2. GENERAZIONE ORARI DI PARTENZA DEGLI ORGANI
# ---------------------------------------------------------
def genera_orari_partenza_organi(
    organi: List[str],
    inizio_giorno: datetime.datetime = datetime.datetime.combine(datetime.date.today(), datetime.time(0, 0))
) -> Dict[str, List[datetime.datetime]]:
    """
    Genera gli orari di partenza per una lista di organi.
    Dominio: [0, 0.5, ..., 24h]
    Numero orari per organo: Random tra 10 e 35.
    """
    
    # Definizione del dominio temporale [0, 24h] a passi di 30 min
    # 24 ore * 60 min = 1440 min. 
    # Usiamo +1 nel range per includere l'estremo superiore (1440 min = 24:00).
    pool_minuti = list(range(0, (24 * 60) + 1, 30))
    
    risultato = {}

    for organo in organi:
        # Scelta numero di orari (tra 10 e 35)
        n_scelte = random.randint(10, 35)
        
        # Campionamento casuale senza ripetizioni
        scelte_minuti = random.sample(pool_minuti, n_scelte)
        
        # Ordinamento cronologico
        scelte_minuti.sort()
        
        # Conversione in datetime
        lista_orari = []
        for m in scelte_minuti:
            orario = inizio_giorno + datetime.timedelta(minutes=m)
            lista_orari.append(orario)
            
        risultato[organo] = lista_orari

    return risultato

# ---------------------------------------------------------
#  TEST GENERAZIONE ORARI
# ---------------------------------------------------------
print("=== TEST 1: Orari Inizio Intervento ===",end='\n')
print("\nT = [", end=' ')
orari_intervento = genera_orari_possibili_inizio_intervento()

for i, orario in enumerate(orari_intervento):
    print(f"t_{i}: {orario.strftime('%H:%M')}", end=' ')
print("]")

print("\n=== TEST 2: Orari Partenza Organi ===")
orari_organi = genera_orari_partenza_organi(ORGANI)

for organo, orari in orari_organi.items():
    # Calcoliamo quante partenze ci sono
    num_partenze = len(orari)
    # Formattiamo per la stampa
    str_orari = [o.strftime("%H:%M") for o in orari]
    
    print(f"\n--- {organo} (Peso: {PESO_ORGANI.get(organo, 'N/A')}) ---")
    print(f"Totale partenze: {num_partenze}")
    print(f"Primi 3 orari: {str_orari[:3]}")
    print(f"Ultimi 3 orari: {str_orari[-3:]}")

=== TEST 1: Orari Inizio Intervento ===

T = [ t_0: 00:00 t_1: 00:30 t_2: 01:00 t_3: 02:30 ]

=== TEST 2: Orari Partenza Organi ===

--- Cuore (Peso: 10.0) ---
Totale partenze: 10
Primi 3 orari: ['00:00', '07:00', '10:00']
Ultimi 3 orari: ['18:00', '19:00', '21:00']

--- Polmoni (Peso: 9.0) ---
Totale partenze: 12
Primi 3 orari: ['01:30', '02:30', '09:30']
Ultimi 3 orari: ['22:30', '23:00', '00:00']

--- Rene_sx (Peso: 2.0) ---
Totale partenze: 28
Primi 3 orari: ['00:00', '00:30', '01:00']
Ultimi 3 orari: ['21:30', '22:30', '23:00']

--- Rene_dx (Peso: 2.0) ---
Totale partenze: 28
Primi 3 orari: ['00:30', '01:00', '02:00']
Ultimi 3 orari: ['21:30', '23:00', '00:00']

--- Fegato (Peso: 2.0) ---
Totale partenze: 16
Primi 3 orari: ['00:00', '01:00', '01:30']
Ultimi 3 orari: ['20:00', '22:00', '23:30']


In [ ]:
import random
import datetime
from typing import List, Dict

def genera_orari_partenza_organi(
    organi: List[str],
    inizio_giorno: datetime.datetime = datetime.datetime.combine(datetime.date.today(), datetime.time(0, 0))
) -> Dict[str, List[datetime.datetime]]:
    """
    Genera gli orari di partenza per una lista di organi.
    
    Args:
        organi: Lista di identificativi (es. ['Cuore', 'Fegato', ...])
        inizio_giorno: Datetime di riferimento per l'inizio della giornata (t=0)
        
    Returns:
        Dizionario { 'Nome_Organo': [datetime_1, datetime_2, ...] }
    """
    
    # 1. Definizione del dominio temporale [0, 23.5h] a passi di 30 min
    # 23.5 ore * 60 min = 1410 min. +1 per includere il limite superiore (23.5h esatte).
    pool_minuti = list(range(0, (23 * 60) + 31, 30))
    
    risultato = {}

    for organo in organi:
        # 2. Scelta numero di orari (tra 10 e 35)
        n_scelte = random.randint(10, 35)
        
        # 3. Campionamento casuale senza ripetizioni
        scelte_minuti = random.sample(pool_minuti, n_scelte)
        
        # 4. Ordinamento cronologico
        scelte_minuti.sort()
        
        # 5. Conversione in datetime
        lista_orari = []
        for m in scelte_minuti:
            orario = inizio_giorno + datetime.timedelta(minutes=m)
            lista_orari.append(orario)
            
        risultato[organo] = lista_orari

    return risultato

# ---------------------------------------------------------
# Esempio di utilizzo
# ---------------------------------------------------------


orari_totali = genera_orari_partenza_organi(organi)

# Stampa formattata dei risultati
print("Esempio Orari di Partenze organi Generati:")
for organo, orari in orari_totali.items():
    print(f"\n--- {organo} ({len(orari)} partenze) ---")
    # Mostriamo solo i primi 5 e l'ultimo per brevità
    str_orari = [o.strftime("%H:%M") for o in orari]
    print(f"Orari: {str_orari}")
